In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import molsysmt as msm
import nglview as nv
import warnings
warnings.filterwarnings('ignore')

# Working with NGLView

In [ ]:
molsys = msm.convert([nv.datafiles.GRO, nv.datafiles.XTC], to_form='molsysmt.MolSys')

In [ ]:
view = nv.show_molsysmt(molsys, standardize=True, viewer='NGLView')

In [ ]:
view

In [ ]:
msm.info(view)

In [ ]:
msm.get(view, element='group', indices=[81, 82, 83], name=True)

In [ ]:
msm.get(view, element='system', n_structures=True)

In [ ]:
msm.get(view, element='atom', selection='atom_name=="CA"', coordinates=True)

In [ ]:
msm.get(view, element='system', box=True) # Warning: only 1 frame

In [ ]:
msm.is_composed_of(view, proteins=1)

In [ ]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"')

In [ ]:
msm.select(view, selection='atom_name=="CA" and group_name=="LYS"', to_syntax='nglview')

In [ ]:
msm.select(view, element='group', selection='group_name=="LYS"', to_syntax='nglview')

In [ ]:
contact_map = msm.structure.get_contacts(view, selection='molecule_type=="protein" and atom_name=="CA"', threshold='12 angstroms')

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(contact_map[10], cmap='Greys', origin='lower')
plt.show()

In [ ]:
msm.convert(view, to_form='string:aminoacids1')

In [ ]:
molsys_A = msm.build.build_peptide(['AceAlaNME',{'forcefield':'AMBER14', 'implicit_solvent':'OBC1'}])
molsys_B = msm.structure.translate(molsys_A, translation='[0.5, 0.0, 0.0] nm')

In [ ]:
view1 = msm.convert(molsys_A, to_form='nglview.NGLWidget')
view2 = msm.view(molsys_B)

In [ ]:
view = msm.merge([view1, view2])

In [ ]:
msm.info(view)

In [ ]:
view = msm.concatenate_frames([view1, view2])

In [ ]:
msm.info(view)